## Manipolazione di un file GTF (Gene Transfer Format) attraverso la libreria `Pandas`

#### 1) Importare `Pandas`

In [ ]:
import pandas as pd

#### 2) Leggere il file GTF

    df = pd.read_csv(gtf_file_name, sep='\t', header = None)

In [ ]:
df = pd.read_csv('./input.gtf', sep='\t', header = None)

In [ ]:
df

**NB**: `read_csv()` ha un attributo `names` che permette di specificare la lista dei nomi delle colonne del data frame restituito.

#### 3) Cambiare i nomi delle colonne

I nomi delle colonne devono essere:
- reference
- source
- feature
- start
- end
- score
- strand
- frame
- attributes

In [ ]:
replace_dict = {0 : 'reference', 1 : 'source', 2 : 'feature', 3 : 'start', 4 : 'end', 5 : 'score', 6 : 'strand', 7 : 'frame', 8 : 'attributes'}
df.rename(columns = replace_dict, inplace = True)

In [ ]:
df

#### 4) Eliminare le colonne `source` e `score` e sostituire l'identificatore `ENm006` con l'identificatore `ENCODE_REGION` in tutti i campi della colonna `reference`

In [ ]:
df.drop(['source', 'score'], axis=1, inplace = True)

Tre alternative per sostituire `ENm006` con `ENCODE_REGION`:

In [ ]:
df['reference'].replace('ENm006', 'ENCODE_REGION', inplace = True)

In [ ]:
df['reference'] = df['reference'].apply(lambda x: 'ENCODE_REGION')

In [ ]:
df['reference'] = 'ENCODE_REGION'

In [ ]:
df

#### 5) Sostituire la colonna degli attributi con le due colonne  `transcript` e `gene`

La colonne `transcript` e `gene` dovranno contenere solo l'ID del trascritto e del gene.

In [ ]:
import re

In [ ]:
df['gene'] = ''
df['transcript'] = ''

In [ ]:
for (index, record) in df.iterrows():
    transcript_id = re.search('transcript_id\s+(.+?);', record['attributes']).group(1).replace('"', '')
    gene_id = re.search('gene_id\s+(.+?);', record['attributes']).group(1).replace('"', '')
    df.loc[index, 'transcript'] =  transcript_id
    df.loc[index, 'gene'] =  gene_id

In [ ]:
df.drop('attributes', axis=1, inplace=True)

In [ ]:
df

#### 6) Aggiungere la colonna `length` contenente la lunghezza della feature

In [ ]:
df['length'] = df['end'] - df['start'] + 1

Reindicizzazione delle colonne:

In [ ]:
df = df.reindex(columns = ['reference', 'feature', 'start', 'end', 'length', 'strand', 'frame', 'gene', 'transcript'])

In [ ]:
df

#### 7) Rimuovere tutte le features di lunghezza minore o uguale a 6 basi

In [ ]:
df = df[df['length'] > 6]

Alternativa con il metodo `drop()`:

In [ ]:
df.drop(df[df.length <= 6].index, axis=0, inplace = True)

In [ ]:
df

#### 8) Ottenere un data frame ordinato per coordinate crescenti delle features

In [ ]:
df.sort_values('start', ascending = True)

#### 9) Ottenere un data frame ordinato per lunghezza crescente delle features

In [ ]:
df.sort_values('length', ascending = True)

#### 10) Determinare la lista dei geni annotati

In [ ]:
list(df['gene'].unique())

In alternativa si può anche determinare il set:

In [ ]:
set(df['gene'])

#### 11) Determinare la lista degli identificatori dei trascritti annotati

In [ ]:
list(df['transcript'].unique())

#### 12) Calcolare la lunghezza media delle features

In [ ]:
df['length'].mean()

#### 13) Determinare la lunghezza minima degli esoni e i trascritti che contengono un esone di lunghezza minima

a) Determinare la lunghezza minina degli esoni

In [ ]:
min_length = df[df.feature == 'exon'].length.min()

In alternativa si può usare la notazione con il punto:

In [ ]:
min_length = df[df['feature'] == 'exon']['length'].min()

b) Estrarre la lista dei trascritti che contengono un esone di lunghezza minima

In [ ]:
mask = (df.feature == 'exon') & (df.length == min_length)
list(df[mask]['transcript'].unique())

#### 14) Contare quanti trascritti sono annotati per il gene `ARHGAP4`

In [ ]:
len(df[df['gene'] == 'ARHGAP4']['transcript'].unique())

#### 15) Estrarre la lista dei geni con strand `+`

In [ ]:
list(df[df['strand'] == '+']['gene'].unique())

#### 16) Estrarre il set degli esoni (distinti) del gene `ATP6AP1`

**NB**: il set deve essere composto da tuple (start, end).

In [ ]:
df_temp = df[(df['gene'] == 'ATP6AP1') & (df['feature'] == 'exon')]
set(zip(df_temp['start'], df_temp['end']))

#### 17) Contare il numero di trascritti del gene `ARHGAP4` che hanno una CDS annotata

In [ ]:
len(df[(df.gene == 'ARHGAP4') & (df.feature == 'CDS')]['transcript'].unique())

#### 18) Estrarre lo strand del gene  `ATP6AP1`

In [ ]:
df[df['gene'] == 'ATP6AP1']['strand'].unique()[0]

#### 19) Estrarre per ogni trascritto del gene  `ATP6AP1` la lista delle tuple (start, end) dei suoi esoni

a) Estrarre il data frame delle sole righe relative alle features `exon` dei trascritti del gene `ATP6AP1` e delle sole colonne `start`, `end` e `transcript`.

In [ ]:
df_temp = df[(df.gene == 'ATP6AP1') & (df.feature == 'exon')][['start', 'end', 'transcript']]

b) Estrarre la lista dei trascritti

In [ ]:
transcript_list = list(df_temp['transcript'].unique())

c) Estrarre per ogni trascritto la lista delle sue features (start, end)

In [ ]:
feature_list_for_transcript = []

for transcript in transcript_list:
    start_transcript = df_temp[df_temp['transcript'] == transcript]['start']
    end_transcript = df_temp[df_temp['transcript'] == transcript]['end']
    feature_list = list(zip(start_transcript, end_transcript))
    feature_list_for_transcript.append(feature_list)

d) Produrre in output per ogni trascritto la lista delle sue features (start, end)

In [ ]:
for transcript in transcript_list:
    print(transcript)
    print(feature_list_for_transcript.pop(0))

#### 20) Contare per ogni gene quante sono le features annotate per ognuno dei tipi presenti nel GTF

In [ ]:
df.groupby(['gene', 'feature'])['transcript'].count()